## excercise 1

In [1]:
import pandas as pd
#read data
df = pd.read_excel('dataProject4.xlsx',header=4,sheet_name='20000-211000')
#setup data in proper column structuere
df=df.iloc[:,:7].join(df.iloc[:,7:].rename(columns={df.iloc[:,7:].columns[i]:f'Volume {2018+i}' for i in range(len(df.iloc[:,7:].columns))}))
#standardise and clean up column names
df=df.rename(columns={df.columns[i]:str.lower(df.columns[i].split('. ')[-1]) for i in range(len(df.columns))})

In [2]:
#check if there are any cells than contain NaN cells in rows non-result rows and fills the remaining cells with result if it is false
print(f'are there any NaN cells in non-result rows? {(df[df.isna().any(axis=1)]['customer classification (crm)']!='Result').any()}')
if (df[df.isna().any(axis=1)]['customer classification (crm)']!='Result').any() == False:
    df=df.fillna('Result')

are there any NaN cells in non-result rows? False


In [3]:
#drop rows with invalid postal codes, there does not seem to be a way to clean these
drop_index=[]
drop_index+=[item for item in df[df['postal code'].str.len()!=7].index]
drop_index

[72909, 72910, 72911]

In [4]:
df[(df=='Not assigned').sum(axis=1)>=2]

,customer number,postal code,customer classification (crm),horeca menu webshop,name webshop,brand name,contents,volume 2018,volume 2019,volume 2020,volume 2021,volume 2022
429,200046,7607 AE,Hotel,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.504,0.0,3.1104
487,200053,1185 XP,Overige Sport,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.000,0.0,0.2592
603,200062,6075 CD,Golf,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.504,0.0,0.0000
787,200073,6702 PB,Studenten,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.288,0.0,0.5184
1129,200078,3524 BN,Zaal / Partycentrum,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.000,0.0,0.1728
...,...,...,...,...,...,...,...,...,...,...,...,...
81339,210929,4463 AJ,Overige Sport,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.000,0.0,0.2592
81532,210955,5706 CT,Café,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.144,0.0,0.0864
81748,210986,3632 AJ,Hockey,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.288,0.0,0.0000
82218,210999,7312 TS,Jeugdherberg,Not assigned,Not assigned,BAVARIA,0.3,0.0,0.0,0.072,0.0,0.2592


## excercise 2

In [1]:
import pandas as pd
df=pd.read_excel('hotelBookings.xlsx')

In [2]:
#all items in the hotel column are resort hotel and all items in the company column are NaN, therefore these will be removed
removed_columns=['hotel','company']
df=df[[item for item in df.columns if item not in removed_columns]]

In [3]:
if df.duplicated().any():
    print("Warning: The data contains duplicate rows.")
    # Remove duplicate rows
    df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found")

Duplicates removed.


In [4]:
if df.isnull().values.any():
    print("Warning: The data contains missing values.")
else:
    print("No missing values found")

In [54]:
print(df.dtypes)

is_canceled                               object
lead_time                                  int64
arrival_date_year                          int64
arrival_date_month                        object
arrival_date_week_number                   int64
arrival_date_day_of_month                  int64
stays_in_weekend_nights                    int64
stays_in_week_nights                     float64
adults                                     int64
children                                   int64
babies                                     int64
meal                                      object
country                                   object
market_segment                            object
distribution_channel                      object
is_repeated_guest                          int64
previous_cancellations                     int64
previous_bookings_not_canceled             int64
reserved_room_type                        object
assigned_room_type                        object
booking_changes     

In [29]:
def convert_dates_to_datetime(date_series, date_format="%m/%d/%Y"):
    return pd.to_datetime(date_series, format=date_format)


date_series = pd.Series(df['reservation_status_date'])
datetime_series = convert_dates_to_datetime(date_series)

print(datetime_series)

0     2015-07-01
1     2015-07-01
2     2015-07-02
3     2015-07-02
4     2015-07-03
         ...    
995   2015-08-13
996   2015-08-13
997   2015-06-22
998   2015-04-06
999   2015-08-15
Name: reservation_status_date, Length: 1000, dtype: datetime64[ns]


In [53]:
import pandas as pd
import numpy as np


# Forward fill for missing values in 'arrival_date_month'
df['arrival_date_month'] = df['arrival_date_month'].fillna(method='ffill')

# Replace 'NULL' and empty strings with NaN in 'country'
df['country'].replace(['NULL', ''], np.nan, inplace=True)

# Fill NaN in 'country' with 'Unknown'
df['country'].fillna('Unknown', inplace=True)

# Fill NaN in 'agent' with -1
df['agent'].fillna(-1, inplace=True)
df['agent'].replace(['NULL', ''], -1, inplace=True)

# Convert 'country' to numeric; this will make invalid entries NaN
df['numeric_check'] = pd.to_numeric(df['country'], errors='coerce')

# Assign NaN to the original column for numeric values
df['country'] = df['country'].where(df['numeric_check'].isna(), np.nan)

# Optionally drop the 'numeric_check' column if you don't need it anymore
df.drop(columns=['numeric_check'], inplace=True)

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(df[['arrival_date_month', 'country', 'agent']].head(10))



Updated DataFrame:
  arrival_date_month  country  agent
0               July      PRT   -1.0
1               July      PRT   -1.0
2               July      GBR   -1.0
3               July      GBR  304.0
4               July      GBR  240.0
5               July      GBR  240.0
6               July  Unknown   -1.0
7               July      PRT  303.0
8               July      PRT  240.0
9               July      PRT   15.0


C:\Users\bruhh\AppData\Local\Temp\ipykernel_42936\1492381899.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['arrival_date_month'] = df['arrival_date_month'].fillna(method='ffill')


## excersice 3

In [8]:
import pandas as pd
df=pd.read_excel('detailedRetail.xlsx')

In [68]:
#function the turns the dataframe into a report
def df_to_report(df):
    df_report=pd.DataFrame({}) 
    #loop over all unique sales managers
    for loop, name in zip(range(len(df['Sales Manager'].unique())), df['Sales Manager'].unique()):
        #loop over all unique categories
        for i,category in zip(range(len(df['Category'].unique())),df['Category'].unique()):
            #assign a name and categories to the current row
            df_report.loc[i+(len(df.Category.unique())+1)*loop,'Sales Manager']=name
            df_report.loc[i+(len(df.Category.unique())+1)*loop,'Category']=category
            #loop over all unique months and fill in the sales per month per category per manager
            for month in df['Month'].unique():
                df_report.loc[i+(len(df.Category.unique())+1)*loop,month]=df[(df['Category']==category) * (df['Month']==month)*(df['Sales Manager']==name)]['Sales'].sum()
            #fill in the total of months sales per category per manager
            df_report.loc[i+(len(df.Category.unique())+1)*loop,'Total of Month']=df[(df['Category']==category)*(df['Sales Manager']==name)]['Sales'].sum()
        #assign a name and total label to total categories row
        df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,'Sales Manager']=name
        df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,'Category']='Total of Category' 
        #loop over all columns containing numbers
        for colm in [item for item in df_report.columns if item != 'Category' and item!= 'Sales Manager']:
            #fill in total of categories per month per manager and total of categories and months per manager
            df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,colm]=sum(df_report.loc[(len(df.Category.unique())+1)*loop:(len(df.Category.unique())+1)*(loop+1)-2,colm])
    #one final loop for the total of managers
    loop=len(df['Sales Manager'].unique())
    #loop over all unique categories
    for i,category in zip(range(len(df['Category'].unique())),df['Category'].unique()):
        #assign a name and categories to the current row
        df_report.loc[i+(len(df.Category.unique())+1)*loop,'Sales Manager']='Total of Sales Manager'
        df_report.loc[i+(len(df.Category.unique())+1)*loop,'Category']=category
        #loop over all unique months and fill in the total manager sales per month per category 
        for month in df['Month'].unique():
            df_report.loc[i+(len(df.Category.unique())+1)*loop,month]=df[(df['Category']==category) * (df['Month']==month)]['Sales'].sum()
        #fill in the total of month and manager sales per category
        df_report.loc[i+(len(df.Category.unique())+1)*loop,'Total of Month']=df[(df['Category']==category)]['Sales'].sum()
    #assign a name and total label to total categories row
    df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,'Sales Manager']='Total of Sales Manager'
    df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,'Category']='Total of Category'
    #loop over all columns containing numbers
    for colm in [item for item in df_report.columns if item != 'Category' and item != 'Sales Manager']:
    #fill in total of categories and manager per month and total of categories and months and managers
        df_report.loc[(len(df.Category.unique())+1)*(loop+1)-1,colm]=sum(df_report.loc[(len(df.Category.unique())+1)*loop:(len(df.Category.unique())+1)*(loop+1)-2,colm])
    return df_report

#devide two item where the numorator is the first element in a series
def div_series(numorator,denumorator):
    return float(numorator.iloc[0])/denumorator

#turn a report into a percentage report
def report_to_percentage(df,df_report):
    df_percentage_categories=pd.DataFrame({})
    df_percentage_manager=pd.DataFrame({})
    df_percentage_months=pd.DataFrame({})
    full_total=df_report[(df_report['Sales Manager']=='Total of Sales Manager')*(df_report['Category']=='Total of Category')]['Total of Month'].iloc[0]
    #loop over a non-sales columns in the original data frame
    for loop,category in zip(range(len(df['Category'].unique())),df['Category'].unique()):
        #mark the row
        df_percentage_categories.loc[loop,'category']=category
        #add the percantage
        df_percentage_categories.loc[loop,'percentage of total categories']=div_series(df_report[(df_report['Sales Manager']=='Total of Sales Manager')*(df_report['Category']==category)]['Total of Month'],full_total)
    #label the total
    df_percentage_categories.loc[loop+1,'category']='total'
    #order the items
    df_percentage_categories=df_percentage_categories.sort_values('percentage of total categories',ascending=False).reset_index(drop=True)
    #add in the total percentage (god i hope its one)
    df_percentage_categories.loc[loop+1,'percentage of total categories']=df_percentage_categories['percentage of total categories'].sum()

    #the rest are the same for the other non-sales columns in the original data frame
    for loop,name in zip(range(len(df['Sales Manager'].unique())),df['Sales Manager'].unique()):
        df_percentage_manager.loc[loop,'name']=name
        df_percentage_manager.loc[loop,'percentage of total managers']=div_series(df_report[(df_report['Sales Manager']==name)*(df_report['Category']=='Total of Category')]['Total of Month'],full_total)
    df_percentage_manager.loc[loop+1,'name']='total'
    df_percentage_manager=df_percentage_manager.sort_values('percentage of total managers',ascending=False).reset_index(drop=True)
    df_percentage_manager.loc[loop+1,'percentage of total managers']=df_percentage_manager['percentage of total managers'].sum()

    for loop,month in zip(range(len(df['Month'].unique())),df['Month'].unique()):
        df_percentage_months.loc[loop,'month']=month
        df_percentage_months.loc[loop,'percentage of total months']=div_series(df_report[(df_report['Sales Manager']=='Total of Sales Manager')*(df_report['Category']=='Total of Category')][month],full_total)
    df_percentage_months.loc[loop+1,'month']='total'
    df_percentage_months=df_percentage_months.sort_values('percentage of total months',ascending=False).reset_index(drop=True)
    df_percentage_months.loc[loop+1,'percentage of total months']=df_percentage_months['percentage of total months'].sum()
    #return the merged data frame
    return df_percentage_categories.join(df_percentage_manager).join(df_percentage_months)

In [69]:
report=df_to_report(df)

In [71]:
percentage_report=report_to_percentage(df,report)

In [73]:
with pd.ExcelWriter('report.xlsx') as writer:  
    report.to_excel(writer, sheet_name='report')
    percentage_report.to_excel(writer, sheet_name='percentage_report')

## excercise 4

In [27]:
# pip install langdetect

In [28]:
import pandas as pd
import langdetect as ld
df=pd.read_excel('tweets.xlsx')

In [29]:
df

,Tweet ID,Country,Date,Tweet,Definitely English,Ambiguous,Definitely Not English,Code-Switched,Ambiguous due to Named Entities,Automatically Generated Tweets
0,434215992731136000,TR,2014-02-14,BugÃ¼n bulusmami lazimdiii,0,0,1,0,0,0
1,285903159434563008,TR,2013-01-01,Volkan konak adami tribe sokar yemin ederim :D,0,0,1,0,0,0
2,285948076496142016,NL,2013-01-01,Bed,1,0,0,0,0,0
3,285965965118824000,US,2013-01-01,I felt my first flash of violence at some fool...,1,0,0,0,0,0
4,286057979831275008,US,2013-01-01,Ladies drink and get in free till 10:30,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
10497,774941788247298048,TR,2016-09-11,"I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...",1,0,0,0,0,1
10498,774951242422480000,PH,2016-09-11,"El Nido, fica ao norte de Palawan, uma das ilh...",0,0,1,0,0,0
10499,774960083721531008,ID,2016-09-11,Alhamdulillah ðŸ˜ŠðŸ‘¨â€ðŸ‘©â€ðŸ‘§â€ðŸ‘§ðŸŽ...,0,0,1,0,0,0
10500,775057244798849024,NG,2016-09-11,Eid-Mubarak @ Bauchi Fedral Lowcost https://t....,0,1,0,0,1,0


In [38]:
#ld.detect is very unreliable :(
tweets_df = pd.DataFrame(df['Tweet'],columns = ['Tweet'])

def detect_language(tweet):

    """
    function to detect a language from tweets dataframe

    args -> tweet: string
        
    returns -> language of tweet
    """
    
    #seed to make avoid fluctuations in the prediction
    ld.DetectorFactory.seed = 20

    #ignore row if it's not a string
    if not isinstance(tweet, str):
        return 'Unknown'
    
    #use langdetect to predict the languages in the tweet
    try:
        return ld.detect(tweet) #or ld.detect_langs(tweet) for more predictions
    except ld.LangDetectException:
        return 'Unknown'

tweets_df['Language'] = 'Unknown'
tweets_df.loc[df['Definitely English'] == 1, 'Language'] = 'en'

mask = tweets_df['Language'] == 'Unknown'
tweets_df.loc[mask, 'Language'] = tweets_df.loc[mask, 'Tweet'].apply(detect_language)

tweets_df

,Tweet,Language
0,BugÃ¼n bulusmami lazimdiii,tr
1,Volkan konak adami tribe sokar yemin ederim :D,tr
2,Bed,en
3,I felt my first flash of violence at some fool...,en
4,Ladies drink and get in free till 10:30,en
...,...,...
10497,"I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...",en
10498,"El Nido, fica ao norte de Palawan, uma das ilh...",pt
10499,Alhamdulillah ðŸ˜ŠðŸ‘¨â€ðŸ‘©â€ðŸ‘§â€ðŸ‘§ðŸŽ...,id
10500,Eid-Mubarak @ Bauchi Fedral Lowcost https://t....,de


In [31]:
test_tweet = ['my name is Lotus', 'mein Name ist Lotus', 'mijn naam is Lotus']
test_df = pd.DataFrame(test_tweet, columns=['Tweet'])

test_df['language'] = test_df['Tweet'].apply(detect_language)
test_df

#shorter texts are going to be predicted wrongly

,Tweet,language
0,my name is Lotus,en
1,mein Name ist Lotus,de
2,mijn naam is Lotus,nl
